In [1]:
import sqlite3
import pandas as pd
import plotly.express as px
import geopandas as gpd
import plotly.io as pio

In [2]:
conn = sqlite3.connect('earthquake.db')
conn

In [3]:
# Tables for wells
wells = pd.read_csv('../data/wells.csv')
wells.to_sql('wells', conn, if_exists='replace', index=False)

well_locations = pd.read_csv('../data/well_locations.csv')
well_locations.to_sql('well_locations', conn, if_exists='replace', index=False)

well_intake = pd.read_csv('../data/well_intake.csv')
well_intake.to_sql('well_intake', conn, if_exists='replace', index=False)

operators = pd.read_csv('../data/operators.csv')
operators.to_sql('operators', conn, if_exists='replace', index=False)

orders = pd.read_csv('../data/orders.csv')
orders.to_sql('orders', conn, if_exists='replace', index=False)

9685

In [4]:
# Tables for earthquakes

events_df1 = pd.read_csv('../data/events1.csv')
events_df2 = pd.read_csv('../data/events2.csv')
combined_events = pd.concat([events_df1, events_df2], ignore_index=True)
combined_events.to_sql('events', conn, if_exists='replace', index=False)

event_locations1_df = pd.read_csv('../data/event_locations1.csv')
event_locations2_df = pd.read_csv('../data/event_locations2.csv')
combined_event_locations = pd.concat([event_locations1_df, event_locations2_df], ignore_index=True)
combined_event_locations.to_sql('event_locations', conn, if_exists='replace', index=False)

magnitude1_df = pd.read_csv('../data/magnitude1.csv')
magnitude2_df = pd.read_csv('../data/magnitude2.csv')
combined_magnitude = pd.concat([magnitude1_df, magnitude2_df], ignore_index=True)
combined_magnitude.to_sql('magnitude', conn, if_exists='replace', index=False)

15580

In [5]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type= 'table';", conn)

,name
0,wells
1,well_locations
2,well_intake
3,operators
4,orders
5,events
6,event_locations
7,magnitude


In [6]:
# Oklahoma earthquake locations
query1 = """SELECT
    el.event_id, el.latitude, el.longitude
FROM event_locations el
JOIN events e ON e.event_id = el.event_id
WHERE e.year BETWEEN 2009 and 2017
AND state = 'Oklahoma'
"""

result1 = pd.read_sql(query1, conn)
result1 

,event_id,latitude,longitude
0,usp000gt6w,35.1630,-97.8710
1,usp000gt8m,35.7240,-96.9200
2,usp000gtdu,34.5890,-96.3400
3,usp000gu9h,36.3690,-98.0870
4,usp000gucq,34.7350,-96.0360
...,...,...,...
7649,us10006qzl,36.9246,-97.8946
7650,us10006qzw,36.3735,-96.8187
7651,us10006r04,36.4122,-96.8824
7652,us10006r59,36.9393,-97.8960


In [7]:
# Oklahoma injection well locations
query2 = """SELECT DISTINCT
    well_locations.well_number, well_locations.latitude, well_locations.longitude
FROM well_locations
JOIN orders ON orders.well_number = well_locations.well_number
WHERE orders.year BETWEEN 2010 AND 2016
"""
result2 = pd.read_sql(query2, conn)
result2

,well_number,latitude,longitude
0,A-1,33.906562,-97.368262
1,A-1,34.062032,-97.362772
2,A-1,34.208403,-97.368837
3,A-1,34.208860,-97.491591
4,A-1,34.292751,-97.710838
...,...,...,...
7306,26 SWD,36.699122,-98.898064
7307,2-12 SWD,36.740386,-98.770510
7308,2-28 INJ,36.622721,-98.597493
7309,29-B,34.181744,-97.332454


In [8]:
fig1 = px.scatter_map(result1,
            lat=result1.latitude,
            lon=result1.longitude,
            hover_name = 'event_id',
            color_discrete_sequence=['red'])

fig1.show()

In [9]:
fig2 = px.scatter_map(result2,
            lat=result2.latitude,
            lon=result2.longitude,
            hover_name = 'well_number',
            color_discrete_sequence=['blue'])

fig2.show()

In [10]:
fig1.add_trace(fig2.data[0])
fig1.data[0].name = 'Oklahoma earthquakes'
fig1.data[1].name = 'Oklahoma injection wells'
fig1.update_layout(title_text='Overlay of Oklahoma earthquakes and injection wells')
fig1.show()